In [14]:
import json
import numpy as np
import pandas as pd

In [15]:
def solve():
    global iterations, solutions
    n_lim = 4
    l_lim = 4
    for i, pos in enumerate(posisiton_list):
        if nation_list[i] == -1:
            (nation_set, league_set, club_sets, club_leagues) = get_possible_NLC(i)
            for nat in nation_set:
                for lea in league_set:
                    club_set = club_sets[club_leagues.index(lea)]
                    for clu in club_set:
                        nation_list[i] = nat
                        league_list[i] = lea
                        club_list[i] = clu
                        
                        calc_chemistry(i)
                        
                        count_list = [0] * len(posisiton_list)
                        n_count, count_list = nation_count(i, 0, count_list)
                        
                        count_list = [0] * len(posisiton_list)
                        l_count, count_list = league_count(i, 0, count_list)
                        
                        iterations += 1

                        if calc_constrain(n_count, l_count, n_lim, l_lim):                                
                            cont = solve()
                            if cont == 1:
                                return 1
            chemistry_list[i] = 1
            link_matrix[pos["connection_list"], i] = 3
            nation_list[i] = -1
            league_list[i] = -1
            club_list[i] = -1       
            return 0
    solutions += 1
    nations = count_nations()
    leagues = count_leagues()
    clubs = count_clubs()
    print("----", solutions, "----", iterations, "----")
#     print(link_matrix)
#     print(disp_list)
#     print(nation_list, "Nations")
#     print(league_list, "Leagues")
#     print(club_list, "Clubs")
#     print(nations, leagues)
    
    save_list = ([d["formations"][formation_index]['name'], n_lim, l_lim, nations, leagues, clubs, nations+leagues+clubs] + pos_list + 
                 nation_list + league_list + club_list)
    df.loc[len(df), :] = save_list
    
#     a = input("More (y)? ")
#     if a != "y":
#     return 1
#     if solutions == 159:
#         return 1
    return 0

In [16]:
def calc_constrain(n_count, l_count, n_lim, l_lim):
    if sum(chemistry_list) == len(chemistry_list):
        if n_count < n_lim:
            if l_count < l_lim:
                return True
    return False

In [17]:
def nation_count(idx, count, count_list):
    count += 1
    count_list[idx] = 1
    for conn in posisiton_list[idx]["connection_list"]:
        if nation_list[conn] == nation_list[idx] and count_list[conn] == 0:
            count, count_list = nation_count(conn, count, count_list)
    return count, count_list

def league_count(idx, count, count_list):
    count += 1
    count_list[idx] = 1
    for conn in posisiton_list[idx]["connection_list"]:
        if league_list[conn] == league_list[idx] and count_list[conn] == 0:
            count, count_list = league_count(conn, count, count_list)
    return count, count_list

def club_count(idx, count, count_list):
    count += 1
    count_list[idx] = 1
    for conn in posisiton_list[idx]["connection_list"]:
        if club_list[conn] == club_list[idx] and league_list[conn] == league_list[idx] and count_list[conn] == 0:
            count, count_list = club_count(conn, count, count_list)
    return count, count_list

In [18]:
def count_nations():
    n_c = 0
    count_list = [0] * len(posisiton_list)
    for i, pos in enumerate(posisiton_list):
        if count_list[i] == 0:
            n_c += 1
            (c, count_list) = nation_count(i, 0, count_list)
    return n_c

def count_leagues():
    l_c = 0
    count_list = [0] * len(posisiton_list)
    for i, pos in enumerate(posisiton_list):
        if count_list[i] == 0:
            l_c += 1
            (c, count_list) = league_count(i, 0, count_list)
    return l_c

def count_clubs():
    c_c = 0
    count_list = [0] * len(posisiton_list)
    for i, pos in enumerate(posisiton_list):
        if count_list[i] == 0:
            c_c += 1
            (c, count_list) = club_count(i, 0, count_list)
    return c_c

In [19]:
def calc_chemistry(idx):
    for conn in posisiton_list[idx]["connection_list"]:
        if nation_list[conn] != -1:
            link = 0
            if nation_list[conn] == nation_list[idx]:
                link += 1
            if league_list[conn] == league_list[idx]:
                link += 1
                if club_list[conn] == club_list[idx]:
                    link += 1
            link_matrix[idx, conn] = link
            link_matrix[conn, idx] = link
            chemistry_list[conn] = min(1, sum(link_matrix[conn, :]) / len(posisiton_list[conn]["connection_list"]))
    chemistry_list[idx] = min(1, sum(link_matrix[idx, :]) / len(posisiton_list[idx]["connection_list"]))

In [20]:
def get_possible_NLC(idx):
    nation_set = set()
    league_set = set()
    club_leagues = []
    club_sets = []
    for conn in posisiton_list[idx]["connection_list"]:
        if nation_list[conn] != -1:
            nation_set.add(nation_list[conn])
            league_set.add(league_list[conn])
            if league_list[conn] in club_leagues:
                club_sets[club_leagues.index(league_list[conn])].add(club_list[conn])
            else:
                club_leagues.append(league_list[conn])
                club_sets.append({club_list[conn]})
    
    for i, s in enumerate(club_sets):
        club_sets[i] = s
        club_sets[i].add(max(club_sets[i]) + 1)
        
    if not nation_set:
        nation_set.add(0)
    else:
        nation_set.add(max(nation_set) + 1)

    if not league_set:
        league_set.add(0)
        club_leagues.append(0)
        club_sets.append({0})
    else:
        league_set.add(max(league_set) + 1)
        club_leagues.append(max(league_set))
        club_sets.append({0})
    return (nation_set, league_set, club_sets, club_leagues)

In [39]:
formation_index = 8

f = open("../files/formations.json")
d = json.load(f)
posisiton_list = d["formations"][formation_index]["positions_list"]
name = d["formations"][formation_index]['name']
n_pos = len(posisiton_list)
pos_list = []
for i in posisiton_list:
    if 'side' in i:
        pos_list.append(i['side'] + i['position'])
    else:
        pos_list.append(i['position'])
Pcol = []
Ncol = []
Lcol = []
Ccol = []
for i in range(n_pos):
    Pcol.append('pos'+str(i).zfill(2))
    Ncol.append('N'+str(i).zfill(2))
    Lcol.append('L'+str(i).zfill(2))
    Ccol.append('C'+str(i).zfill(2))
col = ['formation', 'Nlim', 'Llim', 'N', 'L', 'C', 'NLC'] + Pcol + Ncol + Lcol + Ccol
df = pd.DataFrame(columns = col)
print(name)

4-3-3


In [40]:
chemistry_list = [1] * len(posisiton_list)
nation_list = [-1] * len(posisiton_list)
league_list = [-1] * len(posisiton_list)
club_list = [-1] * len(posisiton_list)
link_matrix = np.zeros((n_pos,n_pos), dtype=int)
for i, pos in enumerate(posisiton_list):
    link_matrix[pos["connection_list"], i] = 3
    
iterations = 0
solutions = 0
solve()

filename = "C:/Users/Truls/OneDrive/Documents/02 - Privat/Hobby/Python/FUT/hybrid_formations/" + name + ".h5"
df.to_hdf(filename, key = 'df', mode = 'w')
df

---- 1 ---- 442 ----
---- 2 ---- 1339 ----
---- 3 ---- 5049 ----
---- 4 ---- 5064 ----
---- 5 ---- 5095 ----
---- 6 ---- 5110 ----
---- 7 ---- 5139 ----
---- 8 ---- 5155 ----
---- 9 ---- 5171 ----
---- 10 ---- 5184 ----
---- 11 ---- 5185 ----
---- 12 ---- 5189 ----
---- 13 ---- 5200 ----
---- 14 ---- 5201 ----
---- 15 ---- 5205 ----
---- 16 ---- 5227 ----
---- 17 ---- 5228 ----
---- 18 ---- 5232 ----
---- 19 ---- 5247 ----
---- 20 ---- 5273 ----
---- 21 ---- 5274 ----
---- 22 ---- 5278 ----
---- 23 ---- 5330 ----
---- 24 ---- 5346 ----
---- 25 ---- 5362 ----
---- 26 ---- 5375 ----
---- 27 ---- 5376 ----
---- 28 ---- 5380 ----
---- 29 ---- 5391 ----
---- 30 ---- 5392 ----
---- 31 ---- 5396 ----
---- 32 ---- 5952 ----
---- 33 ---- 5957 ----
---- 34 ---- 5958 ----
---- 35 ---- 5962 ----
---- 36 ---- 5989 ----
---- 37 ---- 5994 ----
---- 38 ---- 5995 ----
---- 39 ---- 5999 ----
---- 40 ---- 6034 ----
---- 41 ---- 6060 ----
---- 42 ---- 6079 ----
---- 43 ---- 6080 ----
---- 44 ---- 6084 ---

---- 353 ---- 12248 ----
---- 354 ---- 12259 ----
---- 355 ---- 12260 ----
---- 356 ---- 12264 ----
---- 357 ---- 12820 ----
---- 358 ---- 12825 ----
---- 359 ---- 12826 ----
---- 360 ---- 12830 ----
---- 361 ---- 12857 ----
---- 362 ---- 12862 ----
---- 363 ---- 12863 ----
---- 364 ---- 12867 ----
---- 365 ---- 12902 ----
---- 366 ---- 12928 ----
---- 367 ---- 12947 ----
---- 368 ---- 12948 ----
---- 369 ---- 12952 ----
---- 370 ---- 12953 ----
---- 371 ---- 12957 ----
---- 372 ---- 12963 ----
---- 373 ---- 12968 ----
---- 374 ---- 12969 ----
---- 375 ---- 12973 ----
---- 376 ---- 12985 ----
---- 377 ---- 12995 ----
---- 378 ---- 13005 ----
---- 379 ---- 13010 ----
---- 380 ---- 13011 ----
---- 381 ---- 13015 ----
---- 382 ---- 13030 ----
---- 383 ---- 13051 ----
---- 384 ---- 13056 ----
---- 385 ---- 13057 ----
---- 386 ---- 13061 ----
---- 387 ---- 13075 ----
---- 388 ---- 13076 ----
---- 389 ---- 13085 ----
---- 390 ---- 13095 ----
---- 391 ---- 13096 ----
---- 392 ---- 13100 ----


---- 757 ---- 24831 ----
---- 758 ---- 24835 ----
---- 759 ---- 24858 ----
---- 760 ---- 24868 ----
---- 761 ---- 24878 ----
---- 762 ---- 24893 ----
---- 763 ---- 24914 ----
---- 764 ---- 24924 ----
---- 765 ---- 24934 ----
---- 766 ---- 24949 ----
---- 767 ---- 24968 ----
---- 768 ---- 24979 ----
---- 769 ---- 24988 ----
---- 770 ---- 24989 ----
---- 771 ---- 24998 ----
---- 772 ---- 25004 ----
---- 773 ---- 25005 ----
---- 774 ---- 25014 ----
---- 775 ---- 25020 ----
---- 776 ---- 25030 ----
---- 777 ---- 25038 ----
---- 778 ---- 25039 ----
---- 779 ---- 25043 ----
---- 780 ---- 25044 ----
---- 781 ---- 25048 ----
---- 782 ---- 25054 ----
---- 783 ---- 25059 ----
---- 784 ---- 25060 ----
---- 785 ---- 25064 ----
---- 786 ---- 25081 ----
---- 787 ---- 25091 ----
---- 788 ---- 25096 ----
---- 789 ---- 25097 ----
---- 790 ---- 25101 ----
---- 791 ---- 25116 ----
---- 792 ---- 25137 ----
---- 793 ---- 25142 ----
---- 794 ---- 25143 ----
---- 795 ---- 25147 ----
---- 796 ---- 25189 ----


---- 1130 ---- 35332 ----
---- 1131 ---- 35333 ----
---- 1132 ---- 35337 ----
---- 1133 ---- 35352 ----
---- 1134 ---- 35373 ----
---- 1135 ---- 35378 ----
---- 1136 ---- 35379 ----
---- 1137 ---- 35383 ----
---- 1138 ---- 35406 ----
---- 1139 ---- 35416 ----
---- 1140 ---- 35426 ----
---- 1141 ---- 35441 ----
---- 1142 ---- 35462 ----
---- 1143 ---- 35472 ----
---- 1144 ---- 35482 ----
---- 1145 ---- 35497 ----
---- 1146 ---- 35519 ----
---- 1147 ---- 35529 ----
---- 1148 ---- 35534 ----
---- 1149 ---- 35535 ----
---- 1150 ---- 35539 ----
---- 1151 ---- 35554 ----
---- 1152 ---- 35604 ----
---- 1153 ---- 35605 ----
---- 1154 ---- 35609 ----
---- 1155 ---- 35620 ----
---- 1156 ---- 35621 ----
---- 1157 ---- 35625 ----
---- 1158 ---- 35639 ----
---- 1159 ---- 35640 ----
---- 1160 ---- 35644 ----
---- 1161 ---- 35763 ----
---- 1162 ---- 35774 ----
---- 1163 ---- 35783 ----
---- 1164 ---- 35784 ----
---- 1165 ---- 35793 ----
---- 1166 ---- 35799 ----
---- 1167 ---- 35800 ----
---- 1168 --

---- 1477 ---- 45282 ----
---- 1478 ---- 45286 ----
---- 1479 ---- 45438 ----
---- 1480 ---- 45469 ----
---- 1481 ---- 45493 ----
---- 1482 ---- 45494 ----
---- 1483 ---- 45498 ----
---- 1484 ---- 45499 ----
---- 1485 ---- 45503 ----
---- 1486 ---- 45509 ----
---- 1487 ---- 45514 ----
---- 1488 ---- 45515 ----
---- 1489 ---- 45519 ----
---- 1490 ---- 45536 ----
---- 1491 ---- 45541 ----
---- 1492 ---- 45542 ----
---- 1493 ---- 45546 ----
---- 1494 ---- 45561 ----
---- 1495 ---- 45582 ----
---- 1496 ---- 45587 ----
---- 1497 ---- 45588 ----
---- 1498 ---- 45592 ----
---- 1499 ---- 45611 ----
---- 1500 ---- 45612 ----
---- 1501 ---- 45616 ----
---- 1502 ---- 45617 ----
---- 1503 ---- 45621 ----
---- 1504 ---- 45627 ----
---- 1505 ---- 45628 ----
---- 1506 ---- 45632 ----
---- 1507 ---- 45633 ----
---- 1508 ---- 45637 ----
---- 1509 ---- 45646 ----
---- 1510 ---- 45647 ----
---- 1511 ---- 45651 ----
---- 1512 ---- 45652 ----
---- 1513 ---- 45656 ----
---- 1514 ---- 45685 ----
---- 1515 --

---- 1864 ---- 50877 ----
---- 1865 ---- 50878 ----
---- 1866 ---- 50882 ----
---- 1867 ---- 50932 ----
---- 1868 ---- 50933 ----
---- 1869 ---- 50937 ----
---- 1870 ---- 50938 ----
---- 1871 ---- 50942 ----
---- 1872 ---- 50948 ----
---- 1873 ---- 50953 ----
---- 1874 ---- 50954 ----
---- 1875 ---- 50958 ----
---- 1876 ---- 50970 ----
---- 1877 ---- 50971 ----
---- 1878 ---- 50980 ----
---- 1879 ---- 50990 ----
---- 1880 ---- 50991 ----
---- 1881 ---- 50995 ----
---- 1882 ---- 50996 ----
---- 1883 ---- 51000 ----
---- 1884 ---- 51006 ----
---- 1885 ---- 51007 ----
---- 1886 ---- 51011 ----
---- 1887 ---- 51012 ----
---- 1888 ---- 51016 ----
---- 1889 ---- 51025 ----
---- 1890 ---- 51026 ----
---- 1891 ---- 51030 ----
---- 1892 ---- 51031 ----
---- 1893 ---- 51035 ----
---- 1894 ---- 51126 ----
---- 1895 ---- 51127 ----
---- 1896 ---- 51131 ----
---- 1897 ---- 51132 ----
---- 1898 ---- 51136 ----
---- 1899 ---- 51142 ----
---- 1900 ---- 51143 ----
---- 1901 ---- 51147 ----
---- 1902 --

---- 2200 ---- 58817 ----
---- 2201 ---- 58821 ----
---- 2202 ---- 58822 ----
---- 2203 ---- 58826 ----
---- 2204 ---- 58832 ----
---- 2205 ---- 58837 ----
---- 2206 ---- 58838 ----
---- 2207 ---- 58842 ----
---- 2208 ---- 58859 ----
---- 2209 ---- 58864 ----
---- 2210 ---- 58874 ----
---- 2211 ---- 58879 ----
---- 2212 ---- 58880 ----
---- 2213 ---- 58884 ----
---- 2214 ---- 58899 ----
---- 2215 ---- 58920 ----
---- 2216 ---- 58925 ----
---- 2217 ---- 58926 ----
---- 2218 ---- 58930 ----
---- 2219 ---- 58953 ----
---- 2220 ---- 58958 ----
---- 2221 ---- 58973 ----
---- 2222 ---- 58988 ----
---- 2223 ---- 59009 ----
---- 2224 ---- 59014 ----
---- 2225 ---- 59029 ----
---- 2226 ---- 59044 ----
---- 2227 ---- 59066 ----
---- 2228 ---- 59071 ----
---- 2229 ---- 59081 ----
---- 2230 ---- 59086 ----
---- 2231 ---- 59087 ----
---- 2232 ---- 59091 ----
---- 2233 ---- 59106 ----
---- 2234 ---- 59156 ----
---- 2235 ---- 59167 ----
---- 2236 ---- 59176 ----
---- 2237 ---- 59177 ----
---- 2238 --

---- 2540 ---- 67139 ----
---- 2541 ---- 67165 ----
---- 2542 ---- 67189 ----
---- 2543 ---- 67190 ----
---- 2544 ---- 67194 ----
---- 2545 ---- 67205 ----
---- 2546 ---- 67206 ----
---- 2547 ---- 67210 ----
---- 2548 ---- 67232 ----
---- 2549 ---- 67247 ----
---- 2550 ---- 67248 ----
---- 2551 ---- 67252 ----
---- 2552 ---- 67267 ----
---- 2553 ---- 67293 ----
---- 2554 ---- 67294 ----
---- 2555 ---- 67298 ----
---- 2556 ---- 67322 ----
---- 2557 ---- 67337 ----
---- 2558 ---- 67338 ----
---- 2559 ---- 67342 ----
---- 2560 ---- 67353 ----
---- 2561 ---- 67354 ----
---- 2562 ---- 67358 ----
---- 2563 ---- 67372 ----
---- 2564 ---- 67373 ----
---- 2565 ---- 67377 ----
---- 2566 ---- 67401 ----
---- 2567 ---- 67427 ----
---- 2568 ---- 67451 ----
---- 2569 ---- 67452 ----
---- 2570 ---- 67456 ----
---- 2571 ---- 67467 ----
---- 2572 ---- 67468 ----
---- 2573 ---- 67472 ----
---- 2574 ---- 67494 ----
---- 2575 ---- 67509 ----
---- 2576 ---- 67510 ----
---- 2577 ---- 67514 ----
---- 2578 --

,formation,Nlim,Llim,N,L,C,NLC,pos00,pos01,pos02,...,C01,C02,C03,C04,C05,C06,C07,C08,C09,C10
0,4-3-3,4,4,4,4,4,12,GK,RCB,RB,...,0,0,0,0,0,0,0,0,0,0
1,4-3-3,4,4,4,4,5,13,GK,RCB,RB,...,0,1,0,0,0,0,0,0,0,0
2,4-3-3,4,4,5,4,4,13,GK,RCB,RB,...,0,0,0,0,0,0,0,0,0,0
3,4-3-3,4,4,5,4,4,13,GK,RCB,RB,...,0,0,0,0,0,0,0,0,0,0
4,4-3-3,4,4,5,4,5,14,GK,RCB,RB,...,0,0,0,0,0,0,0,1,0,0
5,4-3-3,4,4,5,4,5,14,GK,RCB,RB,...,0,0,0,0,0,0,0,1,0,0
6,4-3-3,4,4,5,4,4,13,GK,RCB,RB,...,0,0,0,0,0,0,0,0,0,0
7,4-3-3,4,4,5,4,5,14,GK,RCB,RB,...,0,0,0,0,0,0,0,0,1,1
8,4-3-3,4,4,5,5,5,15,GK,RCB,RB,...,0,0,0,0,0,0,0,0,0,0
9,4-3-3,4,4,5,4,4,13,GK,RCB,RB,...,0,0,0,0,0,0,0,0,0,0


In [41]:
df[df['C']==df['C'].max()]

,formation,Nlim,Llim,N,L,C,NLC,pos00,pos01,pos02,...,C01,C02,C03,C04,C05,C06,C07,C08,C09,C10
739,4-3-3,4,4,5,4,8,17,GK,RCB,RB,...,1,1,0,0,0,0,1,0,1,2
853,4-3-3,4,4,5,4,8,17,GK,RCB,RB,...,1,2,0,0,0,0,0,0,1,2
860,4-3-3,4,4,5,4,8,17,GK,RCB,RB,...,1,2,0,0,0,0,0,1,0,1
869,4-3-3,4,4,5,4,8,17,GK,RCB,RB,...,1,2,0,0,0,0,0,0,1,2
877,4-3-3,4,4,4,4,8,16,GK,RCB,RB,...,1,2,0,0,0,0,1,1,0,1
888,4-3-3,4,4,5,4,8,17,GK,RCB,RB,...,1,2,0,0,0,0,0,0,1,2
899,4-3-3,4,4,5,4,8,17,GK,RCB,RB,...,1,2,0,0,0,0,0,1,0,1
910,4-3-3,4,4,5,4,8,17,GK,RCB,RB,...,1,2,0,0,0,0,0,0,1,2
912,4-3-3,4,4,5,4,8,17,GK,RCB,RB,...,1,2,0,0,0,0,0,0,1,2
929,4-3-3,4,4,4,4,8,16,GK,RCB,RB,...,1,2,0,0,0,0,0,0,1,2


Add results to pandas<br>
one row for each result<br>
three coloumns for each position (N, L, C) (33 in total)<br>
count unique N, L and C, and add these as coloumns<br>
- how to count clubs?